## 手写SEBlock模块

In [1]:
import torch
import numpy as np
from torch import nn

In [64]:
class SEBlock(nn.Module):
    '''
    手写seblock
    该模块目前只支持4个维度，B,C,H,W
    初始化参数：
    channel：输入层的通道数；
    scal：缩放因子，默认为0.8，在两个mlp之间使用，减少参数防止过拟合，提高泛化能力
    '''
    def __init__(self, channel, ratio=16):
        super(SEBlock, self).__init__()
        self.scal_channel = int(channel // ratio)
        self.block = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=(1,1)),
            nn.Flatten(),
            # 经过全局池化后要将最后两个维度展平，因此加了一个展平层
            nn.Linear(in_features=channel, out_features=self.scal_channel, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=self.scal_channel, out_features=channel, bias=True),
            nn.Sigmoid()
        )
    def forward(self, x):
        # 扩展两个维度后进行scale操作，根据广播机制乘以原来通道内的所有元素
        return x * self.block(x).unsqueeze(dim=-1).unsqueeze(dim=-1)

# 测试seblock输出，随机生成有两个batch，三个通道，尺寸为3*3的矩阵
a = torch.randint(0,10,size=(2,3,3,3)).float()
seblock = SEBlock(3, ratio=1.5)
seblock(a)

tensor([[[[1.9021, 1.9021, 3.1702],
          [5.7064, 2.5362, 0.0000],
          [2.5362, 1.9021, 3.8043]],

         [[0.6861, 0.1715, 0.4288],
          [0.3430, 0.4288, 0.2573],
          [0.5146, 0.5146, 0.4288]],

         [[0.1812, 0.1812, 0.3624],
          [0.3624, 0.3624, 1.4495],
          [1.0871, 1.6307, 1.0871]]],


        [[[5.4363, 1.2081, 5.4363],
          [1.8121, 0.0000, 1.2081],
          [1.2081, 1.2081, 1.2081]],

         [[0.4351, 0.2175, 0.5439],
          [0.3263, 0.2175, 0.5439],
          [0.3263, 0.3263, 0.5439]],

         [[1.7873, 1.5639, 2.0107],
          [0.2234, 0.8936, 0.4468],
          [0.4468, 0.0000, 0.6702]]]], grad_fn=<MulBackward0>)